In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
experiment = "experiment_trafficlights"

### Reading input data

In [3]:
with open(f'traffic/input_files/{experiment}/traffic.json', "r") as read_file:
    drivers_input_data = json.load(read_file)
with open(f'traffic/input_files/{experiment}/drivers.json', "r") as read_file:
    traffic_input_data = json.load(read_file)
with open(f'traffic/input_files/{experiment}/lights.json', "r") as read_file:
    lights_input_data = json.load(read_file)

### Reading output data

In [16]:
agent_data = pd.read_csv(f'traffic/output_files/{experiment}/agent_data.csv')
agent_data.head()

,Step,AgentID,X,Y,Velocity,Current_lane,Is_alive
0,1,0,0.688056,100.000000,[0.68805623 0. ],1,True
1,1,1,0.000000,100.000000,[1.07108856 0. ],1,False
2,1,2,0.452577,166.666667,[0.45257689 0. ],2,True
3,1,3,0.000000,166.666667,[0.42732181 0. ],2,False
4,1,4,1.118439,33.333333,[1.11843941 0. ],0,True


## Flow rate

#### Flow rate: The flow rate is the number of cars that pass a certain point per unit of time.

In [22]:
nodes = []
for node in lights_input_data:
    nodes.append(node["pos"])
nodes

[0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200]

In [32]:
agent_data[agent_data["Is_alive"]==True].groupby("Step")

## Density
#### Density: Traffic density is the number of cars per unit length of the road at any instant. 